In [102]:
import pandas as pd
from datetime import datetime
from plyer import notification
import requests
import sqlite3
import json

In [103]:
conn = sqlite3.connect('coderhouse.db' )

Função responsável por salvar o DataFrame no Banco de dados local.

In [104]:
def salva_bd(df, nome_tabela):
    conn = sqlite3.connect('coderhouse.db')
    df.to_sql(nome_tabela,conn,if_exists='replace',index = False)
    conn.close()
    return None

Função responsável por carregar o DataFrame do banco de dados local.

In [105]:
def carrega_bd(nome_tabela):
    conn = sqlite3.connect('coderhouse.db')
    query = f"select * from {nome_tabela}"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

Função responsável por listar todas as tabelas salvas no banco de dados local.

In [106]:
def lista_tabelas():
    conn = sqlite3.connect('coderhouse.db')
    query = "select name from sqlite_master where type='table'"
    schema = pd.read_sql(query, conn)
    conn.close()
    return schema

In [107]:
urls = [ 
    'https://brasilapi.com.br/api/banks/v1', #bancos ok
    'https://brasilapi.com.br/api/cvm/corretoras/v1', #corretoras ok
    'https://restcountries.com/v3.1/all' #rest countries ok
]

Função responsável por emitir um alerta caso a requisição da API retorne falha.

In [108]:
def Alerta(nivel, base, etapa):
    
    data_atual = datetime.now()
    data_formatada = data_atual.strftime('%d/%m/%d')
    mensagem = (f'Falha no carregamento da base {base} na etapa {etapa} em {data_formatada}')
    title = nivel
    notification.notify(title = title, message= mensagem, timeout= 10)

Função responsável por direcionar as APIs em seus respectivos Data Frames.

In [109]:
def Extraçao():

    global df_bancos, df_corretoras, df_resties

    for url in urls:
        request = requests.get(url)
        response_code = request.status_code
        if response_code == 200:
            if url == 'https://brasilapi.com.br/api/banks/v1':
                df_bancos = pd.DataFrame(request.json())
                
            elif url == 'https://brasilapi.com.br/api/cvm/corretoras/v1':
                df_corretoras = pd.DataFrame(request.json())
                
            elif url == 'https://restcountries.com/v3.1/all':
                df_resties = pd.DataFrame(request.json())
                
            else:
                print('URL fora da base de dados.')
        else:
            nivel = 'ATENÇÃO: Erro Grave'
            Alerta(nivel = nivel, base = 'APIs', etapa = 'Extraçao')

In [110]:
Extraçao()

                                           INICIANDO TRATAMENTO DO DF_BANCOS

In [111]:
df_bancos.head(2)

,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.


In [112]:
df_bancos_mod = df_bancos.copy() #mantendo o df original sem que o mesmo seja alterado pelas modificações posteriores

In [113]:
df_bancos_mod.head(2)

,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.


In [114]:
df_bancos_mod['code'].fillna(0, inplace=True)

In [115]:
df_bancos_mod['code'] = df_bancos_mod['code'].astype(int)

In [116]:
colunas_com_nan = ['name', 'fullName']

In [117]:
df_bancos_mod = df_bancos_mod.dropna(subset=colunas_com_nan)

In [118]:
df_bancos_mod['fullName'] = df_bancos_mod['fullName'].str.upper()

In [119]:
df_bancos_mod['name'] = df_bancos_mod['name'].str.upper()

In [120]:
df_bancos_mod.rename(columns={'ispb': 'ISPB'}, inplace=True)
df_bancos_mod.rename(columns={'name': 'SOCIAL'}, inplace=True)
df_bancos_mod.rename(columns={'code': 'CÓDIGO'}, inplace=True)
df_bancos_mod.rename(columns={'fullName': 'RAZÃO SOCIAL'}, inplace=True)

In [121]:
df_bancos_mod.head(4)

,ISPB,SOCIAL,CÓDIGO,RAZÃO SOCIAL
0,00000000,BCO DO BRASIL S.A.,1,BANCO DO BRASIL S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70,BRB - BANCO DE BRASILIA S.A.
2,00038121,SELIC,0,BANCO CENTRAL DO BRASIL - SELIC
3,00038166,BACEN,0,BANCO CENTRAL DO BRASIL


SALVANDO E ENVIANDO O DF PARA O BANCO DE DADOS LOCAL

In [122]:
salva_bd(df_bancos_mod,'bancos')

In [123]:
lista_tabelas()

,name
0,corretoras
1,resties
2,bancos


In [124]:
bancos_teste = carrega_bd('bancos')

In [125]:
bancos_teste.head(2)

,ISPB,SOCIAL,CÓDIGO,RAZÃO SOCIAL
0,00000000,BCO DO BRASIL S.A.,1,BANCO DO BRASIL S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70,BRB - BANCO DE BRASILIA S.A.


                                       INICIANDO TRATAMENTO DO DF_CORRETORAS

In [126]:
df_corretoras.head(2)

,cnpj,type,nome_social,nome_comercial,status,email,telefone,cep,pais,uf,municipio,bairro,complemento,logradouro,data_patrimonio_liquido,valor_patrimonio_liquido,codigo_cvm,data_inicio_situacao,data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,2005-12-31,4228660.18,2275,2006-10-05,1968-01-15
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",2002-12-31,0.00,3514,2002-10-14,2002-10-14


In [127]:
df_corretoras_mod = df_corretoras.copy()

In [128]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cnpj                      372 non-null    object
 1   type                      372 non-null    object
 2   nome_social               372 non-null    object
 3   nome_comercial            372 non-null    object
 4   status                    372 non-null    object
 5   email                     372 non-null    object
 6   telefone                  372 non-null    object
 7   cep                       372 non-null    object
 8   pais                      372 non-null    object
 9   uf                        372 non-null    object
 10  municipio                 372 non-null    object
 11  bairro                    372 non-null    object
 12  complemento               372 non-null    object
 13  logradouro                372 non-null    object
 14  data_patrimonio_liquido   

In [129]:
df_corretoras_mod['data_patrimonio_liquido']

0      2005-12-31
1      2002-12-31
2      1989-12-31
3      2022-12-31
4      2000-12-31
          ...    
367    2017-12-31
368    2017-12-31
369    2022-12-31
370    1996-12-31
371    2022-12-31
Name: data_patrimonio_liquido, Length: 372, dtype: object

In [130]:
df_corretoras_mod['data_patrimonio_liquido'] = pd.to_datetime(df_corretoras_mod['data_patrimonio_liquido'])

In [131]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   cnpj                      372 non-null    object        
 1   type                      372 non-null    object        
 2   nome_social               372 non-null    object        
 3   nome_comercial            372 non-null    object        
 4   status                    372 non-null    object        
 5   email                     372 non-null    object        
 6   telefone                  372 non-null    object        
 7   cep                       372 non-null    object        
 8   pais                      372 non-null    object        
 9   uf                        372 non-null    object        
 10  municipio                 372 non-null    object        
 11  bairro                    372 non-null    object        
 12  complemento           

In [132]:
df_corretoras_mod['data_patrimonio_liquido'] = df_corretoras_mod['data_patrimonio_liquido'].dt.strftime('%d/%m/%Y')

In [133]:
df_corretoras_mod['data_inicio_situacao']

0      2006-10-05
1      2002-10-14
2      1990-06-12
3      2022-06-20
4      2001-04-11
          ...    
367    2018-08-27
368    2019-09-24
369    1967-12-06
370    1997-02-21
371    1998-02-10
Name: data_inicio_situacao, Length: 372, dtype: object

In [134]:
df_corretoras_mod['data_inicio_situacao'] = pd.to_datetime(df_corretoras_mod['data_inicio_situacao'])

In [135]:
df_corretoras_mod['data_inicio_situacao'] = df_corretoras_mod['data_inicio_situacao'].dt.strftime('%d/%m/%Y')

In [136]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cnpj                      372 non-null    object
 1   type                      372 non-null    object
 2   nome_social               372 non-null    object
 3   nome_comercial            372 non-null    object
 4   status                    372 non-null    object
 5   email                     372 non-null    object
 6   telefone                  372 non-null    object
 7   cep                       372 non-null    object
 8   pais                      372 non-null    object
 9   uf                        372 non-null    object
 10  municipio                 372 non-null    object
 11  bairro                    372 non-null    object
 12  complemento               372 non-null    object
 13  logradouro                372 non-null    object
 14  data_patrimonio_liquido   

In [137]:
df_corretoras_mod['data_registro'] = pd.to_datetime(df_corretoras_mod['data_registro'])

In [138]:
df_corretoras_mod['data_registro'] = df_corretoras_mod['data_registro'].dt.strftime('%d/%m/%Y') 

In [139]:
df_corretoras_mod.head(2)

,cnpj,type,nome_social,nome_comercial,status,email,telefone,cep,pais,uf,municipio,bairro,complemento,logradouro,data_patrimonio_liquido,valor_patrimonio_liquido,codigo_cvm,data_inicio_situacao,data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002


In [140]:
df_corretoras_mod.columns = [col.capitalize() for col in df_corretoras.columns]

In [141]:
df_corretoras_mod.rename(columns={'Cnpj': 'CNPJ'}, inplace=True)

In [142]:
df_corretoras_mod.rename(columns={'Uf': 'UF'}, inplace=True)

In [143]:
df_corretoras_mod.rename(columns={'Cep': 'CEP'}, inplace=True)

In [144]:
df_corretoras_mod.rename(columns={'Email': 'E-mail'}, inplace=True)

In [145]:
df_corretoras_mod.rename(columns={'Pais': 'País'}, inplace=True)

In [146]:
df_corretoras_mod.rename(columns={'Type': 'Tipo'}, inplace=True)

In [147]:
df_corretoras_mod.head(2)

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002


In [148]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CNPJ                      372 non-null    object
 1   Tipo                      372 non-null    object
 2   Nome_social               372 non-null    object
 3   Nome_comercial            372 non-null    object
 4   Status                    372 non-null    object
 5   E-mail                    372 non-null    object
 6   Telefone                  372 non-null    object
 7   CEP                       372 non-null    object
 8   País                      372 non-null    object
 9   UF                        372 non-null    object
 10  Municipio                 372 non-null    object
 11  Bairro                    372 non-null    object
 12  Complemento               372 non-null    object
 13  Logradouro                372 non-null    object
 14  Data_patrimonio_liquido   

In [149]:
linhas_com_missing = df_corretoras_mod[df_corretoras_mod.isna().any(axis=1)]

In [150]:
linhas_com_missing

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
18,99988065000154,CORRETORAS,ALVES FERREIRA CV LTDA,,CANCELADA,,,80020,BRASIL,PR,CURITIBA,,,"RUA MAL.DEODORO, 235 - 2 ANDAR",NaN,,2747,29/04/1993,27/12/1991
70,12865507000197,CORRETORAS,"BRICKELL S.A. CRÉDITO, FINANCIAMENTO E INVESTI...",--,CANCELADA,brickellcfi@brickellcfi.com.br,21989735,4534002,BRASIL,SP,SÃO PAULO,ITAIM BIBI,SALA 1302 E1303,"R JOAQUIM FLORIANO, 466",NaN,,3492,23/01/1995,28/03/1988
228,36864992000142,CORRETORAS,MAF DISTRIBUIDORA DE TITULOS E VALORES MOBILIA...,MAF DTVM,CANCELADA,juridico.admfiduciaria@apexgroup.com,45601000,22250040,,RJ,RIO DE JANEIRO,BOTAFOGO,TORRE PÃO DE AÇUCAR,"PRAIA DE BOTAFOGO, 501/ 5º ANDAR PARTE",NaN,,3751,06/11/2020,06/11/2020


In [151]:
df_corretoras_mod = df_corretoras_mod.dropna()

In [152]:
df_corretoras_mod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 369 entries, 0 to 371
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CNPJ                      369 non-null    object
 1   Tipo                      369 non-null    object
 2   Nome_social               369 non-null    object
 3   Nome_comercial            369 non-null    object
 4   Status                    369 non-null    object
 5   E-mail                    369 non-null    object
 6   Telefone                  369 non-null    object
 7   CEP                       369 non-null    object
 8   País                      369 non-null    object
 9   UF                        369 non-null    object
 10  Municipio                 369 non-null    object
 11  Bairro                    369 non-null    object
 12  Complemento               369 non-null    object
 13  Logradouro                369 non-null    object
 14  Data_patrimonio_liquido   369 n

In [153]:
df_corretoras_mod.head(5)

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002
2,10664027000132,CORRETORAS,ABERTURA CCVM LTDA,ABERTURA CCVM LTDA,CANCELADA,,,50010240,BRASIL,PE,RECIFE,,,R DO IMP.D.PEDRO II 239/CJ.102,31/12/1989,5995252.29,329,12/06/1990,08/07/1986
3,44527444000155,CORRETORAS,ABN AMRO CORRETORA DE T-TULOS E VALORES MOBILI...,,EM FUNCIONAMENTO NORMAL,canal.comunicacao@br.abnamro.com,30737417,4551010,,SP,SÃO PAULO,VILA OLIMPIA,"11¦ ANDAR, CJ. 111","RUA FIDÛNCIO RAMOS, N¦ 302, TORRE B",31/12/2022,3729687.41,3794,20/06/2022,20/06/2022
4,33819590000138,CORRETORAS,AÇÃO S.A. CORRETORA DE VALORES E CÂMBIO,ACAO SA CVC,CANCELADA,,5095755,20040040,BRASIL,RJ,RIO DE JANEIRO,CENTRO,,"PRAÇA OLAVO BILAC, 28/ SL. 608",31/12/2000,291862.38,1112,11/04/2001,19/04/1968


SALVANDO E ENVIANDO O DF PARA O BANCO DE DADOS LOCAL

In [154]:
salva_bd(df_corretoras_mod,'corretoras')

In [155]:
lista_tabelas()

,name
0,resties
1,bancos
2,corretoras


In [156]:
corretora_teste = carrega_bd('corretoras')

In [157]:
corretora_teste.head(2)

,CNPJ,Tipo,Nome_social,Nome_comercial,Status,E-mail,Telefone,CEP,País,UF,Municipio,Bairro,Complemento,Logradouro,Data_patrimonio_liquido,Valor_patrimonio_liquido,Codigo_cvm,Data_inicio_situacao,Data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,31/12/2005,4228660.18,2275,05/10/2006,15/01/1968
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL CORRETORA,CANCELADA,regina.tkatch@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",31/12/2002,0.00,3514,14/10/2002,14/10/2002


                                         INICIANDO TRATAMENTO DO DF_RESTIES

In [158]:
df_resties.head(2)

,name,tld,cca2,ccn3,cca3,independent,status,unMember,currencies,idd,capital,altSpellings,region,subregion,languages,translations,latlng,landlocked,area,demonyms,flag,maps,population,car,timezones,continents,flags,coatOfArms,startOfWeek,capitalInfo,cioc,borders,gini,fifa,postalCode
0,"{'common': 'British Indian Ocean Territory', '...",[.io],IO,086,IOT,False,officially-assigned,False,"{'USD': {'name': 'United States dollar', 'symb...","{'root': '+2', 'suffixes': ['46']}",[Diego Garcia],[IO],Africa,Eastern Africa,{'eng': 'English'},{'ara': {'official': 'إقليم المحيط الهندي البر...,"[-6.0, 71.5]",False,60.0,"{'eng': {'f': 'Indian', 'm': 'Indian'}}",🇮🇴,{'googleMaps': 'https://goo.gl/maps/bheNucgekV...,3000,"{'signs': ['GB'], 'side': 'right'}",[UTC+06:00],[Asia],"{'png': 'https://flagcdn.com/w320/io.png', 'sv...",{},monday,"{'latlng': [-7.3, 72.4]}",NaN,NaN,NaN,NaN,NaN
1,"{'common': 'Botswana', 'official': 'Republic o...",[.bw],BW,072,BWA,True,officially-assigned,True,"{'BWP': {'name': 'Botswana pula', 'symbol': 'P'}}","{'root': '+2', 'suffixes': ['67']}",[Gaborone],"[BW, Republic of Botswana, Lefatshe la Botswana]",Africa,Southern Africa,"{'eng': 'English', 'tsn': 'Tswana'}","{'ara': {'official': 'جمهورية بوتسوانا', 'comm...","[-22.0, 24.0]",True,582000.0,"{'eng': {'f': 'Motswana', 'm': 'Motswana'}, 'f...",🇧🇼,{'googleMaps': 'https://goo.gl/maps/E364KeLy6N...,2351625,"{'signs': ['BW'], 'side': 'left'}",[UTC+02:00],[Africa],"{'png': 'https://flagcdn.com/w320/bw.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-24.63, 25.9]}",BOT,"[NAM, ZAF, ZMB, ZWE]",{'2015': 53.3},BOT,NaN


In [159]:
df_resties_mod = df_resties.copy()

In [160]:
df_resties_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 35 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          250 non-null    object 
 1   tld           249 non-null    object 
 2   cca2          250 non-null    object 
 3   ccn3          249 non-null    object 
 4   cca3          250 non-null    object 
 5   independent   249 non-null    object 
 6   status        250 non-null    object 
 7   unMember      250 non-null    bool   
 8   currencies    247 non-null    object 
 9   idd           250 non-null    object 
 10  capital       246 non-null    object 
 11  altSpellings  250 non-null    object 
 12  region        250 non-null    object 
 13  subregion     245 non-null    object 
 14  languages     249 non-null    object 
 15  translations  250 non-null    object 
 16  latlng        250 non-null    object 
 17  landlocked    250 non-null    bool   
 18  area          250 non-null    

In [161]:
valores_ausentes_por_coluna = df_resties_mod.isna().sum()

In [162]:
print(valores_ausentes_por_coluna)

name              0
tld               1
cca2              0
ccn3              1
cca3              0
independent       1
status            0
unMember          0
currencies        3
idd               0
capital           4
altSpellings      0
region            0
subregion         5
languages         1
translations      0
latlng            0
landlocked        0
area              0
demonyms          1
flag              0
maps              0
population        0
car               0
timezones         0
continents        0
flags             0
coatOfArms        0
startOfWeek       0
capitalInfo       0
cioc             44
borders          85
gini             83
fifa             44
postalCode      101
dtype: int64


In [163]:
df_resties_mod.columns = [col.capitalize() for col in df_resties_mod.columns]

In [164]:
df_resties_mod.head(2)

,Name,Tld,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Area,Demonyms,Flag,Maps,Population,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo,Cioc,Borders,Gini,Fifa,Postalcode
0,"{'common': 'British Indian Ocean Territory', '...",[.io],IO,086,IOT,False,officially-assigned,False,"{'USD': {'name': 'United States dollar', 'symb...","{'root': '+2', 'suffixes': ['46']}",[Diego Garcia],[IO],Africa,Eastern Africa,{'eng': 'English'},{'ara': {'official': 'إقليم المحيط الهندي البر...,"[-6.0, 71.5]",False,60.0,"{'eng': {'f': 'Indian', 'm': 'Indian'}}",🇮🇴,{'googleMaps': 'https://goo.gl/maps/bheNucgekV...,3000,"{'signs': ['GB'], 'side': 'right'}",[UTC+06:00],[Asia],"{'png': 'https://flagcdn.com/w320/io.png', 'sv...",{},monday,"{'latlng': [-7.3, 72.4]}",NaN,NaN,NaN,NaN,NaN
1,"{'common': 'Botswana', 'official': 'Republic o...",[.bw],BW,072,BWA,True,officially-assigned,True,"{'BWP': {'name': 'Botswana pula', 'symbol': 'P'}}","{'root': '+2', 'suffixes': ['67']}",[Gaborone],"[BW, Republic of Botswana, Lefatshe la Botswana]",Africa,Southern Africa,"{'eng': 'English', 'tsn': 'Tswana'}","{'ara': {'official': 'جمهورية بوتسوانا', 'comm...","[-22.0, 24.0]",True,582000.0,"{'eng': {'f': 'Motswana', 'm': 'Motswana'}, 'f...",🇧🇼,{'googleMaps': 'https://goo.gl/maps/E364KeLy6N...,2351625,"{'signs': ['BW'], 'side': 'left'}",[UTC+02:00],[Africa],"{'png': 'https://flagcdn.com/w320/bw.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-24.63, 25.9]}",BOT,"[NAM, ZAF, ZMB, ZWE]",{'2015': 53.3},BOT,NaN


In [165]:
df_resties_mod['Name'] = [pais['common']for pais in df_resties_mod['Name']]

In [166]:
df_resties_mod['Postalcode']

0                                                    NaN
1                                                    NaN
2              {'format': '#####', 'regex': '^(\d{5})$'}
3               {'format': '####', 'regex': '^(\d{4})$'}
4               {'format': '####', 'regex': '^(\d{4})$'}
                             ...                        
245    {'format': 'CP ####', 'regex': '^(?:CP)*(\d{4}...
246         {'format': '#####', 'regex': '^(986\d{2})$'}
247                                                  NaN
248             {'format': '####', 'regex': '^(\d{4})$'}
249                                                  NaN
Name: Postalcode, Length: 250, dtype: object

In [167]:
df_resties_mod = df_resties_mod.drop('Postalcode', axis=1)

In [168]:
pd.options.display.max_columns = None

In [169]:
df_resties_mod.head(2)

,Name,Tld,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Area,Demonyms,Flag,Maps,Population,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo,Cioc,Borders,Gini,Fifa
0,British Indian Ocean Territory,[.io],IO,086,IOT,False,officially-assigned,False,"{'USD': {'name': 'United States dollar', 'symb...","{'root': '+2', 'suffixes': ['46']}",[Diego Garcia],[IO],Africa,Eastern Africa,{'eng': 'English'},{'ara': {'official': 'إقليم المحيط الهندي البر...,"[-6.0, 71.5]",False,60.0,"{'eng': {'f': 'Indian', 'm': 'Indian'}}",🇮🇴,{'googleMaps': 'https://goo.gl/maps/bheNucgekV...,3000,"{'signs': ['GB'], 'side': 'right'}",[UTC+06:00],[Asia],"{'png': 'https://flagcdn.com/w320/io.png', 'sv...",{},monday,"{'latlng': [-7.3, 72.4]}",NaN,NaN,NaN,NaN
1,Botswana,[.bw],BW,072,BWA,True,officially-assigned,True,"{'BWP': {'name': 'Botswana pula', 'symbol': 'P'}}","{'root': '+2', 'suffixes': ['67']}",[Gaborone],"[BW, Republic of Botswana, Lefatshe la Botswana]",Africa,Southern Africa,"{'eng': 'English', 'tsn': 'Tswana'}","{'ara': {'official': 'جمهورية بوتسوانا', 'comm...","[-22.0, 24.0]",True,582000.0,"{'eng': {'f': 'Motswana', 'm': 'Motswana'}, 'f...",🇧🇼,{'googleMaps': 'https://goo.gl/maps/E364KeLy6N...,2351625,"{'signs': ['BW'], 'side': 'left'}",[UTC+02:00],[Africa],"{'png': 'https://flagcdn.com/w320/bw.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-24.63, 25.9]}",BOT,"[NAM, ZAF, ZMB, ZWE]",{'2015': 53.3},BOT


In [170]:
df_resties_mod['Currencies']

0      {'USD': {'name': 'United States dollar', 'symb...
1      {'BWP': {'name': 'Botswana pula', 'symbol': 'P'}}
2      {'UYU': {'name': 'Uruguayan peso', 'symbol': '...
3            {'MKD': {'name': 'denar', 'symbol': 'den'}}
4               {'EUR': {'name': 'Euro', 'symbol': '€'}}
                             ...                        
245    {'USD': {'name': 'United States dollar', 'symb...
246        {'XPF': {'name': 'CFP franc', 'symbol': '₣'}}
247      {'WST': {'name': 'Samoan tālā', 'symbol': 'T'}}
248    {'ETB': {'name': 'Ethiopian birr', 'symbol': '...
249    {'XCD': {'name': 'Eastern Caribbean dollar', '...
Name: Currencies, Length: 250, dtype: object

In [171]:
df_resties_mod.head(1)

,Name,Tld,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Area,Demonyms,Flag,Maps,Population,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo,Cioc,Borders,Gini,Fifa
0,British Indian Ocean Territory,[.io],IO,086,IOT,False,officially-assigned,False,"{'USD': {'name': 'United States dollar', 'symb...","{'root': '+2', 'suffixes': ['46']}",[Diego Garcia],[IO],Africa,Eastern Africa,{'eng': 'English'},{'ara': {'official': 'إقليم المحيط الهندي البر...,"[-6.0, 71.5]",False,60.0,"{'eng': {'f': 'Indian', 'm': 'Indian'}}",🇮🇴,{'googleMaps': 'https://goo.gl/maps/bheNucgekV...,3000,"{'signs': ['GB'], 'side': 'right'}",[UTC+06:00],[Asia],"{'png': 'https://flagcdn.com/w320/io.png', 'sv...",{},monday,"{'latlng': [-7.3, 72.4]}",NaN,NaN,NaN,NaN


In [172]:
df_resties_mod['Currencies'].dropna()

0      {'USD': {'name': 'United States dollar', 'symb...
1      {'BWP': {'name': 'Botswana pula', 'symbol': 'P'}}
2      {'UYU': {'name': 'Uruguayan peso', 'symbol': '...
3            {'MKD': {'name': 'denar', 'symbol': 'den'}}
4               {'EUR': {'name': 'Euro', 'symbol': '€'}}
                             ...                        
245    {'USD': {'name': 'United States dollar', 'symb...
246        {'XPF': {'name': 'CFP franc', 'symbol': '₣'}}
247      {'WST': {'name': 'Samoan tālā', 'symbol': 'T'}}
248    {'ETB': {'name': 'Ethiopian birr', 'symbol': '...
249    {'XCD': {'name': 'Eastern Caribbean dollar', '...
Name: Currencies, Length: 247, dtype: object

In [173]:
# Função para extrair a chave 'name' do dicionário
def extrair_nome_moeda(dicionario):
    if isinstance(dicionario, dict):
        for chave, valor in dicionario.items():
            if 'name' in valor:
                return valor['name']
    return None
        

# Extrair e atribuir a chave 'name' à coluna 'currencies'
df_resties_mod['Currencies'] = df_resties_mod['Currencies'].apply(extrair_nome_moeda)

# Renomear a coluna para 'Currencies_Name'
df_resties_mod = df_resties_mod.rename(columns={'Currencies': 'Currencies_Name'})


In [174]:
df_resties_mod.head(1)

,Name,Tld,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies_Name,Idd,Capital,Altspellings,Region,Subregion,Languages,Translations,Latlng,Landlocked,Area,Demonyms,Flag,Maps,Population,Car,Timezones,Continents,Flags,Coatofarms,Startofweek,Capitalinfo,Cioc,Borders,Gini,Fifa
0,British Indian Ocean Territory,[.io],IO,086,IOT,False,officially-assigned,False,United States dollar,"{'root': '+2', 'suffixes': ['46']}",[Diego Garcia],[IO],Africa,Eastern Africa,{'eng': 'English'},{'ara': {'official': 'إقليم المحيط الهندي البر...,"[-6.0, 71.5]",False,60.0,"{'eng': {'f': 'Indian', 'm': 'Indian'}}",🇮🇴,{'googleMaps': 'https://goo.gl/maps/bheNucgekV...,3000,"{'signs': ['GB'], 'side': 'right'}",[UTC+06:00],[Asia],"{'png': 'https://flagcdn.com/w320/io.png', 'sv...",{},monday,"{'latlng': [-7.3, 72.4]}",NaN,NaN,NaN,NaN


In [175]:
df_resties_mod['Borders'].fillna('NA', inplace=True)

In [176]:
colunas_com_listas = df_resties_mod.columns[df_resties_mod.applymap(type).eq(list).any()]

In [177]:
colunas_com_listas

Index(['Tld', 'Capital', 'Altspellings', 'Latlng', 'Timezones', 'Continents',
       'Borders'],
      dtype='object')

In [178]:
for colunas in colunas_com_listas:
    df_resties_mod[colunas] = df_resties_mod[colunas].apply(json.dumps)

In [179]:
df_resties_mod['Continents'] = df_resties_mod['Continents'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[2:-2])

In [180]:
df_resties_mod['Capital'] = df_resties_mod['Capital'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[2:-2])

In [181]:
df_resties_mod['Latlng'] = df_resties_mod['Latlng'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [182]:
df_resties_mod['Borders'] = df_resties_mod['Borders'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [183]:
df_resties_mod['Timezones'] = df_resties_mod['Timezones'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [184]:
df_resties_mod['Capital'] = df_resties_mod['Capital'].str.strip('"')

In [185]:
df_resties_mod = df_resties_mod.drop(columns=['Translations','Demonyms','Languages','Gini','Capitalinfo','Idd','Car','Tld','Altspellings','Maps','Flags','Coatofarms'])

In [186]:

def converter_dict_para_list(value):
    if isinstance(value, dict):
        return list(value)  # Converte o dicionário em uma string
    else:
        return value  # Mantém outros valores inalterados


In [187]:
df_resties_mod['Timezones'] = df_resties_mod['Timezones'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x[1:-1])

In [188]:
df_resties_mod.head(2)

,Name,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies_Name,Capital,Region,Subregion,Latlng,Landlocked,Area,Flag,Population,Timezones,Continents,Startofweek,Cioc,Borders,Fifa
0,British Indian Ocean Territory,IO,086,IOT,False,officially-assigned,False,United States dollar,Diego Garcia,Africa,Eastern Africa,"-6.0, 71.5",False,60.0,🇮🇴,3000,UTC+06:00,Asia,monday,NaN,NA,NaN
1,Botswana,BW,072,BWA,True,officially-assigned,True,Botswana pula,Gaborone,Africa,Southern Africa,"-22.0, 24.0",True,582000.0,🇧🇼,2351625,UTC+02:00,Africa,monday,BOT,"""NAM"", ""ZAF"", ""ZMB"", ""ZWE""",BOT


In [189]:
df_resties_mod['Cioc'] = df_resties_mod['Cioc'].fillna('Not_Apply')

In [190]:
valores_ausentes_por_coluna = df_resties_mod.isna().sum()

In [191]:
valores_ausentes_por_coluna

Name                0
Cca2                0
Ccn3                1
Cca3                0
Independent         1
Status              0
Unmember            0
Currencies_Name     3
Capital             0
Region              0
Subregion           5
Latlng              0
Landlocked          0
Area                0
Flag                0
Population          0
Timezones           0
Continents          0
Startofweek         0
Cioc                0
Borders             0
Fifa               44
dtype: int64

In [192]:
df_resties_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             250 non-null    object 
 1   Cca2             250 non-null    object 
 2   Ccn3             249 non-null    object 
 3   Cca3             250 non-null    object 
 4   Independent      249 non-null    object 
 5   Status           250 non-null    object 
 6   Unmember         250 non-null    bool   
 7   Currencies_Name  247 non-null    object 
 8   Capital          250 non-null    object 
 9   Region           250 non-null    object 
 10  Subregion        245 non-null    object 
 11  Latlng           250 non-null    object 
 12  Landlocked       250 non-null    bool   
 13  Area             250 non-null    float64
 14  Flag             250 non-null    object 
 15  Population       250 non-null    int64  
 16  Timezones        250 non-null    object 
 17  Continents      

In [193]:
mapeamento = {True: 'Yes', False: 'No'}

In [194]:
colunas_a_tratar = ['Independent', 'Unmember']
df_resties_mod[colunas_a_tratar] = df_resties_mod[colunas_a_tratar].replace(mapeamento)

In [195]:
df_resties_mod.head(2)

,Name,Cca2,Ccn3,Cca3,Independent,Status,Unmember,Currencies_Name,Capital,Region,Subregion,Latlng,Landlocked,Area,Flag,Population,Timezones,Continents,Startofweek,Cioc,Borders,Fifa
0,British Indian Ocean Territory,IO,086,IOT,No,officially-assigned,No,United States dollar,Diego Garcia,Africa,Eastern Africa,"-6.0, 71.5",False,60.0,🇮🇴,3000,UTC+06:00,Asia,monday,Not_Apply,NA,NaN
1,Botswana,BW,072,BWA,Yes,officially-assigned,Yes,Botswana pula,Gaborone,Africa,Southern Africa,"-22.0, 24.0",True,582000.0,🇧🇼,2351625,UTC+02:00,Africa,monday,BOT,"""NAM"", ""ZAF"", ""ZMB"", ""ZWE""",BOT


SALVANDO E ENVIANDO O DF PARA O BANCO DE DADOS LOCAL

In [196]:

salva_bd(df_resties_mod,'resties')

In [197]:
lista_tabelas()

,name
0,bancos
1,corretoras
2,resties


In [198]:
resties_teste = carrega_bd('resties')

In [199]:
resties_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             250 non-null    object 
 1   Cca2             250 non-null    object 
 2   Ccn3             249 non-null    object 
 3   Cca3             250 non-null    object 
 4   Independent      249 non-null    object 
 5   Status           250 non-null    object 
 6   Unmember         250 non-null    object 
 7   Currencies_Name  247 non-null    object 
 8   Capital          250 non-null    object 
 9   Region           250 non-null    object 
 10  Subregion        245 non-null    object 
 11  Latlng           250 non-null    object 
 12  Landlocked       250 non-null    int64  
 13  Area             250 non-null    float64
 14  Flag             250 non-null    object 
 15  Population       250 non-null    int64  
 16  Timezones        250 non-null    object 
 17  Continents      